In [4]:
import sqlite3

# Connect to sqllite
connection = sqlite3.connect('student.db')
# Create a cursor object to inser record, create table, retreive
cursor = connection.cursor()

# Create the table
table_info = """
Create table STUDENT(NAME VARCHAR(25), CLASS VARCHAR(25),
SECTION VARCHAR(25), MARKS INT)"""

cursor.execute(table_info)

# Insert some more records into the table
cursor.execute('''Insert into STUDENT values ('Alice', 'Machine Learning', 'B', 85)''')  
cursor.execute('''Insert into STUDENT values ('Bob', 'Artificial Intelligence', 'A', 92)''')  
cursor.execute('''Insert into STUDENT values ('Charlie', 'Cyber Security', 'C', 78)''')  
cursor.execute('''Insert into STUDENT values ('David', 'Cloud Computing', 'B', 88)''')  
cursor.execute('''Insert into STUDENT values ('Emma', 'Software Engineering', 'A', 95)''')  

# Display all the records
print('The inserted records are')

data = cursor.execute('''Select * from Student''')

for row in data:
    print(row)

# Close the cursor
cursor.close()

The inserted records are
('Alice', 'Machine Learning', 'B', 85)
('Bob', 'Artificial Intelligence', 'A', 92)
('Charlie', 'Cyber Security', 'C', 78)
('David', 'Cloud Computing', 'B', 88)
('Emma', 'Software Engineering', 'A', 95)


In [ ]:
import os
import streamlit as st

from langchain_google_genai import GoogleGenerativeAIEmbeddings
import google.generativeai as genai
import sqlite3

genai.configure(api_key = os.getenv('GOOGLE_API_KEY'))

# Function to load Google Gemini model and provide sql query as response
def get_gemini_response(question, prompt):
    model = genai.get_model('gemini-pro')
    response = model.generate_content([prompt, question])
    return response.text

# Function to retrieve query from the sql database
def read_sql_query(sql, db):
    conn = sqlite3.connect(db)
    cur = conn.cursor()
    cur.execute(sql)
    rows = cur.fetchall()
    conn.commit()
    conn.close()
    for row in rows:
        print(row)
    return rows


## Define Your Prompt
prompt=[
    """
    You are an expert in converting English questions to SQL query!
    The SQL database has the name STUDENT and has the following columns - NAME, CLASS, 
    SECTION \n\nFor example,\nExample 1 - How many entries of records are present?, 
    the SQL command will be something like this SELECT COUNT(*) FROM STUDENT ;
    \nExample 2 - Tell me all the students studying in Data Science class?, 
    the SQL command will be something like this SELECT * FROM STUDENT 
    where CLASS="Data Science"; 
    also the sql code should not have ``` in beginning or end and sql word in output

    """


]

## Streamlit App

st.set_page_config(page_title="I can Retrieve Any SQL query")
st.header("Gemini App To Retrieve SQL Data")

question = st.text_input("Input: ", key = "input")

submit=st.button("Ask the question")

# if submit is clicked
if submit:
    response = get_gemini_response(question,prompt)
    print(response)
    response = read_sql_query(response,"student.db")
    st.subheader("The Response is")
    for row in response:
        print(row)
        st.header(row)